For help implementing the adaboost algorithm, I used these resource to ensure I had the right parameters: https://towardsdatascience.com/adaboost-for-dummies-breaking-down-the-math-and-its-equations-into-simple-terms-87f439757dcf



In [96]:
import pandas as pd
import numpy as np 
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from pandas import Series, DataFrame
import string


letter = pd.read_csv('letter-recognition.data',header=None)
#this is a multiclass dataset. Going to have to do some weird things to it.

X1 = letter.drop(letter.columns[0], axis=1)
y1 = letter.iloc[:,0]
#y1 = pd.get_dummies(letter.iloc[:,0])
y1.head()
X_train, X_test, y_train, y_test = train_test_split(X1, y1)

In [101]:
credit = pd.read_csv('german.data-numeric',sep='\s+',header=None)
#credit.head()
#good or bad credit, last column
X2 = credit.iloc[:,:-1] # Features 
y2 = credit.iloc[:,-1] # Target 
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2)

In [115]:
spam = pd.read_csv('spambase.data',header=None)
#spam.head()
#last column is spam (1) or not spam (0)
X3 = spam.iloc[:,:-1] # Features 
y3 = spam.iloc[:,-1] # Target 
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3)

#AdaBoost Algorithm 

In [114]:

#The input of your function is the training and test sets, as well as the number of rounds of boosting T.

#The function should then return the predictions of the final combined classifier on the given training and test examples, as well as the training and test #error rate of the combined classifier following each of the T rounds.
from matplotlib import pyplot as plt


def AdaBoost_scratch(X,y, M=10, learning_rate = 1):
    #Initialization of utility variables
    N = len(y)
    estimator_list, y_predict_list, estimator_error_list, estimator_weight_list, sample_weight_list = [], [],[],[],[]

    #Initialize the sample weights
    sample_weight = np.ones(N) / N
    sample_weight_list.append(sample_weight.copy())

    #For m = 1 to M
    for m in range(M):   

        #Fit a classifier
        estimator = DecisionTreeClassifier(max_depth = 1)
        estimator.fit(X, y, sample_weight=sample_weight)
        y_predict = estimator.predict(X)

        #Misclassifications
        incorrect = (y_predict != y)

        #Estimator error
        estimator_error = np.mean( np.average(incorrect, weights=sample_weight, axis=0))
        
        #Boost estimator weights
        estimator_weight =  learning_rate * np.log((1. - estimator_error) / estimator_error)

        #Boost sample weights
        sample_weight *= np.exp(estimator_weight * incorrect * ((sample_weight > 0) | (estimator_weight < 0)))

        #Save iteration values
        estimator_list.append(estimator)
        y_predict_list.append(y_predict.copy())
        estimator_error_list.append(estimator_error.copy())
        estimator_weight_list.append(estimator_weight.copy())
        sample_weight_list.append(sample_weight.copy())
        


    #Convert to np array for convenience   
    estimator_list = np.asarray(estimator_list)
    y_predict_list = np.asarray(y_predict_list)
    estimator_error_list = np.asarray(estimator_error_list)
    estimator_weight_list = np.asarray(estimator_weight_list)
    sample_weight_list = np.asarray(sample_weight_list)

    preds = (np.array([np.sign((y_predict_list[:,point] * estimator_weight_list).sum()) for point in range(N)]))
    print('Accuracy of train = ', (preds == y).sum() / N) 

    return estimator_list, y_predict_list, estimator_error_list, estimator_weight_list, sample_weight_list

def predictions(X,y, estimators, s_weights, e_weights):
    N = len(y)
    y_preds = []
    for m,e_w in zip(estimators,e_weights):
        estimator = DecisionTreeClassifier(max_depth = 1)
        estimator.fit(X, y)
        preds = estimator.predict(X)
        y_preds.append(preds)

    y_preds = np.asarray(y_preds)
    test_preds = (np.array([np.sign((y_preds[:,point] * e_weights).sum()) for point in range(N)]))
    print('Accuracy of test = ', (preds == y).sum() / N) 
    

E, Y, EE, EW, SW = AdaBoost_scratch(X_train2,y_train2)
predictions(X_test2,y_test2,E,SW,EW)


Accuracy of train =  0.6933333333333334
Accuracy of test =  0.72


In [116]:
E, Y, EE, EW, SW = AdaBoost_scratch(X_train3,y_train3)
predictions(X_test3,y_test3,E,SW,EW)

Accuracy of train =  0.6933333333333334
Accuracy of test =  0.72
